In [1]:
import json
import time

from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError, APIInvalidSSLCertificateError
from config import OWM_API_key as key, connection_port, user, loc_host, remo_host, password, socket_path

API_key = key
owm = OWM(API_key) # the OWM object
print(type(owm))

<class 'pyowm.weatherapi25.owm25.OWM25'>


In [2]:
def set_location_and_get_current(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    
    global owm
    
    try:
        obs = owm.weather_at_zip_code(f'{code}', 'us')
    except APIInvalidSSLCertificateError:
        print(f'except first try in set_location(): APIInvalidSSLCertificateError with zipcode {code}...trying again')
        try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')        
            print('this time it worked')
        except APIInvalidSSLCertificateError:
            print('except on second try in set_location(): APIInvalidSSLCertificateError - reestablishing the OWM object and trying again.')
            try:
                owm = OWM(API_key)    # the OWM object
                obs = owm.weather_at_zip_code(f'{code}', 'us')
                print('this time it worked')
            except APIInvalidSSLCertificateError:
                print('....and again... this time I am just gonna return.')
                return(f'the time is {time.time()}')
    except APICallTimeoutError:
        print('caught APICallTimeoutError on first try in set_location()...trying again')
        time.sleep(5)
        try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')
            print('this time it worked')
        except APICallTimeoutError:
            time.sleep(5)
            print('caught APICallTimeoutError on second try in set_location()...trying again')
            try:
                obs = owm.weather_at_zip_code(f'{code}', 'us')
                print('this time it worked')
            except APICallTimeoutError:
                print(f'could not get past the goddamn api call for {code}! Returning with nothing but shame this time.')
                return(f'the time is {time.time()}')
    current = json.loads(obs.to_JSON()) # dump all that weather data to a json object
    return(current)

def five_day(zlat, zlon):
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code
        :type five_day: dict
    '''
    
    global owm
    
    try:
        forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    except APIInvalidSSLCertificateError:
        print(f'except on first try in five_day(): APIInvalidSSLCertificateError ...trying again')
        try:
            forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
            print('this time it worked')
        except APIInvalidSSLCertificateError:
            print('except on second try five_day(): APIInvalidSSLCertificateReeor... reestablish the OWM object and try again.')
            try:
                owm = OWM(API_key)    # the OWM object
                forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
                print('this time it worked')
            except APIInvalidSSLCertificateError:
                print('....and again... this time I am just gonna return.')
                return(f'the time is {time.time()}')
    except APICallTimeoutError:
        print('caught APICallTimeoutError on fritst try in five_day(). trying again...')
        time.sleep(.5)
        try:
            forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
        except APICallTimeoutError:
            print('caught APICallTimeoutError on second try in five_day(). trying again...')
            time.sleep(.5)
            try:
                forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
            except APICallTimeoutError:
                print('caught APICallTimeoutError on third try in five_day()...returning without another try.')
                return(f'the time is {time.time()}')
    # Get the actual forecasts and add them to a list to be passed on while changing the key 'reference_time' to 'instant'
    forecast = forecaster.get_forecast()
    forecast = json.loads(forecast.to_JSON())
    for f in forecast['weathers']:
        f['instant'] = f.pop('reference_time')        
    return(forecast)

def sort_casts(forecasts, code, client):
    ''' Take the array of forecasts from the five day forecast and sort them into the documents of the instants collection.
        
        :param forecasts: the forecasts from five_day()-  They come in a list of 40, one for each of every thrid hour over five days
        :type forecasts: list- expecting a list of forecasts
        :param code: the zipcode
        :type code: string
        :param client: the mongodb client
        :type client: MongoClient
    '''
    db = client.test
    col = db.instant
    # update each forecast
    for forecast in forecasts:
        forecast['location'] = f['Location']['coordinates']
        forecast['time_to_instant'] = f['weathers'][0]['instant'] - f['reception_time']
        # remove all the fields that will give null values
        forecast.pop('sunset_time')
        forecast.pop('sunrise_time')
        forecast['temperature'].pop('temp_kf')
        forecast.pop('weather_icon_name')
        forecast.pop('visibility_distance')
        forecast.pop('dewpoint')
        forecast.pop('humidex')
        forecast.pop('heat_index')st['heat_index']}
        # now find the document that has that code and that ref_time
        # This should change to the instants collection, find a singel instant specified by zip and the
        # forecast ref_time, and finally append the forecast to the forecasts object
        filter_by_zip_and_inst = {'zipcode':code, 'instant':forecast['reference_time']}
        filters = filter_by_zip_and_inst
        add_forecast_to_instant = {'$push': {'forecasts': forecast}}
        updates = add_forecast_to_instant
        col.find_one_and_update(filters, updates, upsert=True, return_document=ReturnDocument.AFTER)

        
        
        # def make_instants(code, forcasts):
#     ''' create the initial instant object from the five_day weather objects using the zipcode and instant_time
    
#         :param weather: the OWM-reported five-day forecast object
#         :param type: json object
#         :return:
#     '''
#     ### use the forecast variable to make the instants using the reference_time, location
#     instant = {}
    

In [25]:
code = '27006'
current = set_location_and_get_current(code)
location = current['Location']['coordinates']
current.pop('Location')
current['instant'] = 10800*(current['Weather']['reference_time']//10800 + 1) # add the instant time, derived from the reference_time
current

{'reception_time': 1582168580,
 'Weather': {'reference_time': 1582168580,
  'sunset_time': 1582153583,
  'sunrise_time': 1582113916,
  'clouds': 1,
  'rain': {},
  'snow': {},
  'wind': {'speed': 1.5, 'deg': 350},
  'humidity': 28,
  'pressure': {'press': 1030, 'sea_level': None},
  'temperature': {'temp': 277.24,
   'temp_kf': None,
   'temp_max': 279.26,
   'temp_min': 274.82},
  'status': 'Clear',
  'detailed_status': 'clear sky',
  'weather_code': 800,
  'weather_icon_name': '01n',
  'visibility_distance': 16093,
  'dewpoint': None,
  'humidex': None,
  'heat_index': None},
 'instant': 1582178400}

In [24]:
### the beginnings of the forecast handling function. This is transforming the forecasts and 
f = five_day(location['lon'], location['lat'])
forecasts = f['weathers']
# transform the forecast items in the five_day forecast
for forecast in forecasts:
    forecast['location'] = f['Location']['coordinates']
    forecast['time_to_instant'] = f['weathers'][0]['instant'] - f['reception_time']
    # remove all the fields that will give null values
    forecast.pop('sunset_time')
    forecast.pop('sunrise_time')
    forecast['temperature'].pop('temp_kf')
    forecast.pop('weather_icon_name')
    forecast.pop('visibility_distance')
    forecast.pop('dewpoint')
    forecast.pop('humidex')
    forecast.pop('heat_index')
    ### Now for the next step take the forecast and find the instant object with the same instant time and add it to the forecasts ###
forecasts[0]

{'clouds': 70,
 'rain': {},
 'snow': {},
 'wind': {'speed': 5.83, 'deg': 15},
 'humidity': 96,
 'pressure': {'press': 1031, 'sea_level': 1031},
 'temperature': {'temp': 228.4, 'temp_max': 228.4, 'temp_min': 228.4},
 'status': 'Clouds',
 'detailed_status': 'broken clouds',
 'weather_code': 803,
 'instant': 1582167600,
 'location': {'lon': 0.0, 'lat': 0.0},
 'time_to_instant': 3222}

In [ ]:
import json
import os
import time
import urllib
# import schedule
import dns

from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from pyowm.exceptions.api_response_error import NotFoundError
from pyowm.exceptions.api_call_error import APICallTimeoutError, APIInvalidSSLCertificateError
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError, OperationFailure
from urllib.parse import quote
from config import OWM_API_key as key, connection_port, user, password, socket_path

global zipcode
global obs
global zid
global zlon
global zlat
global client

API_key = key
port = connection_port
owm = OWM(API_key)    # the OWM object
password = quote(password)    # url encode the password for the mongodb uri
uri = "mongodb+srv://%s:%s@%s" % (user, password, socket_path)
print(uri)

def make_zip_list(state):
    ''' Make a list of zip codes in the specified state.
        Read zip codes from downloadable zip codes csv file available at https://www.unitedstateszipcodes.org/
        
        :param state: the two-letter abreviation of the state whose zip codes you'd like listed
        :type state: string
        
        :returns success_zips: list of zip codes that OWM has records for
        :type success_zips: list
    '''
    import pandas as pd
    from pyowm.exceptions.api_response_error import NotFoundError
    
    global owm
    
    successes = 0
    exceptions = 0
    success_zips = []
    fail_zips = []
    
    df = pd.read_csv("resources/zip_code_database.csv")
    # Make a datafram from the rows with the specified state and write the 'zip' column to a list 
    zip_list = df.loc[lambda df: df['state'] == f'{state.upper()}']['zip'].tolist()
    for zipp in zip_list:
        if int(zipp) > 10000:
            try:
                set_location(zipp)
                success_zips.append(zipp)
                successes+=1
                time.sleep(.9)
            except APIInvalidSSLCertificateError:
                print(f'except first try: APIInvalidSSLCertificateError with zipcode {zipp}...trying again')
                set_location(zipp)
                success_zips.append(zipp)
                successes+=1
                print('this time it worked')
                time.sleep(.9)
            except APIInvalidSSLCertificateError:
                print('except on second try: APIInvalidSSLCertificateError - reestablishing the OWM object and trying again.')
                owm = OWM(API_key)    # the OWM object
                set_location(zipp)
                success_zips.append(zipp)
                successes+=1
                print('this time it worked')
                time.sleep(.9)
            except APIInvalidSSLCertificateError:
                print('....and again... this time I am just gonna pass.')
                pass
            except NotFoundError:
                print("except", f'NotFoundError with zipcode {zipp}')
                fail_zips.append(zipp)
                exceptions+=1
                pass
    write_list_to_file(success_zips, f'resources/success_zips{state}.csv')
    write_list_to_file(fail_zips, f'resources/fail_zips{state}.csv')
    print(f'successes = {successes}; exceptions = {exceptions}, all written to files')
    return(success_zips)

def write_list_to_file(zip_list, filename):
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        z_list.write(",".join(str(z) for z in zip_list))
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def set_location_and_get_current(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    
    global owm
    
    try:
        obs = owm.weather_at_zip_code(f'{code}', 'us')
    except APIInvalidSSLCertificateError:
        print(f'except first try in set_location(): APIInvalidSSLCertificateError with zipcode {zipp}...trying again')
        try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')        
            print('this time it worked')
        except APIInvalidSSLCertificateError:
            print('except on second try in set_location(): APIInvalidSSLCertificateError - reestablishing the OWM object and trying again.')
            try:
                owm = OWM(API_key)    # the OWM object
                obs = owm.weather_at_zip_code(f'{code}', 'us')
                print('this time it worked')
            except APIInvalidSSLCertificateError:
                print('....and again... this time I am just gonna return.')
                return(f'the time is {time.time()}')
    except APICallTimeoutError:
        print('caught APICallTimeoutError on first try in set_location()...trying again')
        time.sleep(5)
        try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')
            print('this time it worked')
        except APICallTimeoutError:
            time.sleep(5)
            print('caught APICallTimeoutError on second try in set_location()...trying again')
            try:
            obs = owm.weather_at_zip_code(f'{code}', 'us')
            print('this time it worked')
            except APICallTimeoutError:
                print(f'could not get past the goddamn api call for {code}! Returning with nothing but shame this time.')
                return(f'the time is {time.time()}')
    current = json.loads(obs.to_JSON()) # dump all that weather data to a json object
    return(current)

def five_day(zlat, zlon):
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code
        :type five_day: dict
    '''
    
    global owm
    
    try:
        forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    except APIInvalidSSLCertificateError:
        print(f'except on first try in five_day(): APIInvalidSSLCertificateError ...trying again')
        try:
            forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
            print('this time it worked')
        except APIInvalidSSLCertificateError:
        print('except on second try five_day(): APIInvalidSSLCertificateReeor... reestablish the OWM object and try again.')
            try:
                owm = OWM(API_key)    # the OWM object
                forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
                print('this time it worked')
            except APIInvalidSSLCertificateError:
                print('....and again... this time I am just gonna return.')
                return(f'the time is {time.time()}')
    except APICallTimeoutError:
        print('caught APICallTimeoutError on fritst try in five_day(). trying again...')
        time.sleep(.5)
        try:
            forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
        except APICallTimeoutError:
            print('caught APICallTimeoutError on second try in five_day(). trying again...')
            time.sleep(.5)
            try:
                forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
                print('caught APICallTimeoutError on third try in five_day()...returning without another try.')
                return(f'the time is {time.time()}')
    # Get the actual forecasts and add them to a list to be passed on while changing the key 'reference_time' to 'instant'
    forecast = forecaster.get_forecast()
    forecast = json.loads(forecast.to_JSON())
    for f in forecast['weathers']:
        f['instant'] = f.pop('reference_time')        
    return(forecast)


# def get_weather(codes, loc_host, port):
def get_weather(codes, uri):
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
#     client = check_db_access(loc_host, port)
    client = check_db_access(uri)
    for code in codes:
        instant = int(10800*(time.time()//10800)) # the instant that each forecast refers to
        weather = {}
        forecast = {}
        set_location(code)
        weather.update({
                     'zipcode': code,
                     'current': current(),
                     'instant': instant
                    })
        load(weather, client, 'weather')
        forecast.update({
                     'zipcode': code,
                     'five_day': five_day()
                    })
        load(forecast, client, 'forecast')
    client.close()
    return


# def check_db_access(loc_host, port):
def check_db_access(uri):
    ''' Check the database connection and return the client
    
        :param host: the database host
        :type host: string
        :param port: the database connection port
        :type port: int
        :param uri: the conneciton uri for the remote mongo database
        :type uri: sting
    '''
#     client = MongoClient(host=host, port=port)
    client = MongoClient(uri)    
    try:
        client.admin.command('ismaster')
    except ConnectionFailure:
        print("Server not available")
        return
    return(client)


def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable.
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param code: the zip code associated with the data from the list codes
        :type code: sting
    '''
    collection = data
    directory = os.getcwd()
    save_path = os.path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return


def load(data, client, name):
    # because this function is used in a loop, I want to name the variable, {name}, to be whatever the diciontary's
    # name happens to be. That is getting set to a colleciton name 
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
    if type(data) == dict:
        database = client.OWM
        # name = name
        try:
            col = Collection(database, name)
            # db = client.forcast
            # col = db.code
            col.insert_one(data)
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return
